In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/TweetsCOV19.tsv',sep = '\t',skiprows = list(range(4467891,8935782)),usecols=range(11),iterator=True, chunksize=4460000)
df = pd.concat(data, ignore_index=True)
data2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/TweetsCOV19.tsv',sep = '\t',skiprows = list(range(4467891)),usecols=range(11),iterator=True, chunksize=4460000)
df2 = pd.concat(data2, ignore_index=True)
frame = pd.concat([df,df2])

In [ ]:
frame = pd.concat([pd.read_csv('/content/drive/My Drive/Colab Notebooks/TweetsCOV19.tsv',sep = '\t',skiprows = list(range(4467891,8935782)),usecols=range(11),header=None),pd.read_csv('/content/drive/My Drive/Colab Notebooks/TweetsCOV19.tsv',sep = '\t',skiprows = list(range(4467891)),usecols=range(11),header=None)])

frame.columns = ['Tweet Id', 'Username', 'Timestamp', 'Followers', 'Friends', 'Retweets', 'Favorites', 'Entities', 'Sentiment', 'Mentions', 'Hashtags']

In [ ]:
frame['Timestamp'] = frame.Timestamp.apply(lambda x: x.split())

In [ ]:
frame['Day'] = frame.Timestamp.apply(lambda x: x[0])

In [ ]:
frame['Month'] = frame.Timestamp.apply(lambda x: x[1])
frame['Date'] = frame.Timestamp.apply(lambda x: x[2])
frame['Time'] = frame.Timestamp.apply(lambda x: x[3].split(':')[0])
frame['Year'] = frame.Timestamp.apply(lambda x: x[5])

In [ ]:
frame['Pos'] = frame.Sentiment.apply(lambda x: int(x.split()[0]))

In [ ]:
frame['Neg'] = frame.Sentiment.apply(lambda x: int(x.split()[1]))

In [ ]:
frame = frame.drop(columns=['Timestamp','Entities','Username','Sentiment'])

In [ ]:
frame.replace('null;', np.NaN,inplace=True)

In [ ]:
frame.Hashtags.loc[pd.notnull(frame.Hashtags)]

In [1]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [ ]:
frame.to_csv('data.csv')
!cp data.csv "drive/My Drive/Colab Notebooks/"

In [2]:
import pandas as pd
import numpy as np
frame = pd.read_csv('/content/drive/My Drive/Colab Notebooks/comp9417/data.csv')
#convert month to number
#convert day of week to number

month_map={'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11 , 'Dec':12}
frame.Month = frame.Month.map(month_map)

day_map={'Mon':1, 'Tue':2, 'Wed':3, 'Thu':4, 'Fri':5, 'Sat':6, 'Sun':7}
frame.Day = frame.Day.map(day_map)

drop_col=['Unnamed: 0', 'Tweet Id','Mentions', 'Hashtags']
frame.drop(drop_col,axis=1,inplace=True)

print(frame.Retweets.unique())
print(len(frame.Retweets.unique()))

Y = frame.pop('Retweets')
X=frame


[    0    13    26 ...  6816  3868 10791]
9441


In [2]:
#VIF
from scipy.linalg.lapack import spotrf, sposv
import numpy as np
import pandas as pd

def variance_inflation_factor(X):
    """
    Calculates the variance inflation factor for each column in a dataframe.
    Input:
        X: Dataframe
    Output:
        Dataframe of variance inflation values for each column.
    """
    isDataframe = type(X) is pd.DataFrame
    if isDataframe:
        columns = X.columns
        X = X.values
    n, p = X.shape

    swap = np.arange(p)
    np.random.shuffle(swap)

    XTX = X.T @ X
    XTX = XTX[swap][:,swap]

    select = np.ones(p, dtype = bool)

    temp = XTX.copy().T
    error = 1
    largest = XTX.diagonal().max() // 2
    add = largest
    maximum = np.finfo(np.float32).max

    while error != 0:
        C, error = spotrf(a = temp)
        if error != 0:
            error -= 1
            select[error] = False
            temp[error, error] += add
            error += 1

            add += np.random.randint(1,30)
            add *= np.random.randint(30,50)
        if add > maximum:
            add = largest

    VIF = np.empty(p, dtype = np.float32)
    means = np.mean(X, axis = 0)[swap]

    for i in range(p):
        curr = select.copy()
        s = swap[i]

        if curr[i] == False:
            VIF[s] = np.inf
            continue
        curr[i] = False

        XX = XTX[curr]
        xtx = XX[:, curr]
        xty = XX[:,i]
        y_x = X[:,s]

        theta_x = sposv(xtx, xty)[1]
        y_hat = X[:,swap[curr]] @ theta_x

        SS_res = y_x-y_hat
        SS_res = np.einsum('i,i', SS_res, SS_res)
        #SS_res = np.sum((y_x - y_hat)**2)

        SS_tot = y_x - means[i]
        SS_tot = np.einsum('i,i', SS_tot, SS_tot)
        #SS_tot = np.sum((y_x - np.mean(y_x))**2)
        if SS_tot == 0:
            R2 = 1
            VIF[s] = np.inf
        else:
            R2 = 1 - (SS_res/SS_tot)
            VIF[s] = 1/(1-R2)
        del XX, xtx, xty, y_x, theta_x, y_hat
    if isDataframe:
        df_vif = pd.DataFrame({"vif": VIF})
        df_vif = df_vif.set_index(columns)
        return df_vif
    return VIF

In [3]:
import pandas as pd, numpy as np, os, gc, matplotlib.pyplot as plt, seaborn as sb, re, warnings, calendar, sys
from copy import deepcopy
import statsmodels.api as sm
from math import log2
from collections import defaultdict
import seaborn as sns; sns.set()
from sklearn import preprocessing

print(X)
model = sm.OLS(Y, X).fit()
print(model.summary())
# A VIF of 1 indicates two variables are not correlated, a VIF between 1 and 5 indicates moderate correlation, and a VIF above 5 indicates high correlation.
vif= variance_inflation_factor(X)
df_coeffs = pd.DataFrame({"Coefficients": model.params, "p": model.pvalues, "vif": vif["vif"]})
# df_coeffs = df_coeffs.drop("const", axis=0)

df_coeffs.style.bar(subset = ["Coefficients", "p", "vif"], align='mid', color=['#d65f5f', '#5fba7d'])

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


         Followers  Friends  Favorites  Day  Month  Date  Time  Year  Pos  Neg
0              619      770          0    1      9    30    22  2019    2   -1
1            36365    19344         17    1      9    30    22  2019    1   -4
2             5018     1933          0    1      9    30    22  2019    2   -1
3             2219      971          0    1      9    30    22  2019    2   -1
4             9009     4943         89    1      9    30    22  2019    2   -1
...            ...      ...        ...  ...    ...   ...   ...   ...  ...  ...
8077789         45      236          0    4      4    30    21  2020    1   -1
8077790          5       74          0    4      4    30    21  2020    3   -1
8077791      46987     4953        125    4      4    30    21  2020    2   -3
8077792      21796    22109          0    4      4    30    21  2020    1   -2
8077793         25      203          0    4      4    30    21  2020    2   -1

[8077794 rows x 10 columns]
                       

,Coefficients,p,vif
Followers,0.000001,0.000000,1.003803
Friends,0.000221,0.000000,0.983077
Favorites,0.220974,0.000000,0.815590
Day,0.098664,0.044378,0.999913
Month,-0.131827,0.000004,0.971134
Date,-0.011523,0.307788,1.009051
Time,-0.127012,0.000000,0.995270
Year,0.005626,0.000000,0.000001
Pos,-1.611387,0.000000,0.934733
Neg,-1.840344,0.000000,0.994306


In [2]:
import time
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,recall_score
from sklearn.ensemble import RandomForestClassifier

print(X.shape,Y.shape)
print(Y.unique())
X_RF=X[:10000]
Y_RF=Y[:10000]
print(len(Y_RF.unique()))
Y_RF = LabelEncoder().fit_transform(Y_RF)
X_RF = StandardScaler().fit_transform(X_RF)

# print(len(X[0]))
# X_RF=X_RF[:10000]
# Y_RF=Y_RF[:10000]



def forest_test(X, Y):
    X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, 
                                                        test_size = 0.30, 
                                                        random_state = 42)
    print(X_Train.shape,X_Test.shape,Y_Train.shape, Y_Test.shape)
    start = time.process_time()
    trainedforest = RandomForestClassifier(n_estimators=500,random_state=1).fit(X_Train,Y_Train)
    print('time to fit RF',time.process_time() - start)
    predictionforest = trainedforest.predict(X_Test)
    print('accuracy score',accuracy_score(Y_Test,predictionforest))
    # print('recall score', recall_score(Y_Test,predictionforest,average='micro'))
    # print(confusion_matrix(Y_Test,predictionforest))
    # print(classification_report(Y_Test,predictionforest))

forest_test(X_RF, Y_RF)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


(8077794, 10) (8077794,)
[    0    13    26 ...  6816  3868 10791]
420
(7000, 10) (3000, 10) (7000,) (3000,)
time to fit RF 25.852219863
accuracy score 0.6233333333333333


In [ ]:
# import joblib
# import pandas as pd
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
# import warnings
# warnings.filterwarnings('ignore', category=FutureWarning)
# warnings.filterwarnings('ignore', category=DeprecationWarning)
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import StandardScaler

# X_RF=X[:10000]
# Y_RF=Y[:10000]
# print(len(Y_RF.unique()))
# Y_RF = LabelEncoder().fit_transform(Y_RF)
# X_RF = StandardScaler().fit_transform(X_RF)

# def print_results(results):
#     print('BEST PARAMS: {}\n'.format(results.best_params_))

#     means = results.cv_results_['mean_test_score']
#     stds = results.cv_results_['std_test_score']
#     for mean, std, params in zip(means, stds, results.cv_results_['params']):
#         print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

# rf = RandomForestClassifier()
# parameters = {
#     'n_estimators': [250,500],
#     # 'max_depth': [8, 16, 32, None]
# }

# print(X_RF.shape,Y_RF.shape)
# cv = GridSearchCV(rf, parameters, cv=5)
# cv.fit(X_RF, Y_RF)

# print_results(cv)


420
(10000, 10) (10000,)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [11]:
#Linear Regression
from sklearn import metrics
import time
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, 
                                                        test_size = 0.30, 
                                                        random_state = 42)
print(X_Train.shape,X_Test.shape,Y_Train.shape, Y_Test.shape)
start = time.process_time()
model=LinearRegression()
model.fit(X_Train,Y_Train)
print('Time to fit model',time.process_time() - start)
predictions=model.predict(X_Test)
print('Return the coefficient of determination R^2 of the prediction.',model.score(X_Test,Y_Test))
print('Mean squared log error MSLE',metrics.mean_squared_log_error(Y_Test,predictions))
# plt.scatter(Y_Test,predictions)
# plt.xlabel("Actual Retweets")
# plt.ylabel("Predicted Retweets")
# plt.show()

#k-fold cross validation
model=LinearRegression()
cv_4_results = cross_val_score(model, X, Y, cv=5)
cv_4_results


(5654455, 10) (2423339, 10) (5654455,) (2423339,)
Time to fit model 1.6940796250000005
Return the coefficient of determination R^2 of the prediction. 0.829673768373932
Mean squared log error MSLE 3.5612113242821137


array([0.7327294 , 0.72533973, 0.85255705, 0.81285665, 0.76812381])

In [8]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import metrics
import time
from sklearn.model_selection import train_test_split
# print(RandomForestClassifier())
# print(RandomForestRegressor())

X_RF=X[:100000]
Y_RF=Y[:100000]

X_Train, X_Test, Y_Train, Y_Test = train_test_split(X_RF, Y_RF, 
                                                        test_size = 0.30, 
                                                        random_state = 42)
print(X_Train.shape,X_Test.shape,Y_Train.shape, Y_Test.shape)
start = time.process_time()
model=RandomForestRegressor()
model.fit(X_Train,Y_Train)
print('Time to fit model',time.process_time() - start)
predictions=model.predict(X_Test)
print('Return the coefficient of determination R^2 of the prediction.',model.score(X_Test,Y_Test))
print('Mean squared log error MSLE',metrics.mean_squared_log_error(Y_Test,predictions))

(70000, 10) (30000, 10) (70000,) (30000,)
Time to fit model 45.455493115000024
Return the coefficient of determination R^2 of the prediction. 0.4157344650943916
Mean squared log error MSLE 0.2517106097499519
